# Modelo Transfomers basado en RoBERTa
### Detección automática de sarcasmos en textos utilizando Transformers
#### Ernesto Francisco Barrionuevo Aragonés 21727751

## Indice
- Preprocesamiento de datos
- Entrenamiento del modelo
- Busqueda de hiperparametros
- Entrenamiento del modelo con los parametros optimizados

## Preprocesamiento de datos

Se descargan las librerías necesarias

In [1]:
!pip install transformers datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00


In [14]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 197.2 kB/s eta 0:00:00


In [18]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.3 MB/s eta 0:00:00


Se importan las librerías necesarias

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import optuna

Se inicia sesión en Hugging Face desde la línea de comandos

In [18]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Se carga el conjunto de datos sobre sarcasmo en español desde Hugging Face previamente cargado.

In [3]:
dataset = load_dataset("Ernesto-1997/Sarcastic_spanish_dataset")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/105 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/314k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15276 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3820 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Texto', 'Sarcasmo'],
        num_rows: 15276
    })
    validation: Dataset({
        features: ['Texto', 'Sarcasmo'],
        num_rows: 3820
    })
})

Se cambia el nombre de la columna de la variable independiente, ya que el modelo Transformers de Hugging Face necesita que este sea el nombre.

In [7]:
dataset = dataset.rename_column("Sarcasmo", "label")

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Texto', 'label'],
        num_rows: 15276
    })
    validation: Dataset({
        features: ['Texto', 'label'],
        num_rows: 3820
    })
})

Se selecciona el modelo preentrenado "roberta-base-bne", especializado en español, y se configura un tokenizador que prepara los textos para que el modelo pueda procesarlos correctamente. 

In [9]:
model_checkpoint = "BSC-TeMU/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Mostramos el tamaño del vocabulario. En este caso es de 50262

In [10]:
tokenizer.vocab_size

50262

Se crea una función que tokeniza los textos del conjunto de datos para que puedan ser procesados por el modelo. Luego, se eliminan las columnas innecesarias y se aplica la función de tokenización a todo el conjunto de datos de manera eficiente, preparando los textos para su uso en el modelo de entrenamiento.

In [11]:
def tokenize_reviews(examples):
    return tokenizer(examples["Texto"], truncation=True)

In [12]:
columns = dataset["train"].column_names
columns.remove("label")
encoded_dataset = dataset.map(tokenize_reviews, batched=True, remove_columns=columns)
encoded_dataset

Map:   0%|          | 0/15276 [00:00<?, ? examples/s]

Map:   0%|          | 0/3820 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 15276
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 3820
    })
})

Se carga el modelo preentrenado para realizar la tarea de clasificación de sarcasmos, configurado para distinguir entre dos categorías diferentes.

In [13]:
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Se define una función que calcula varias métricas de evaluación, incluyendo precisión, recall, F1 y accuracy. Estas métricas se utilizan para evaluar el rendimiento del modelo de clasificación. Las predicciones del modelo se comparan con las etiquetas reales, y se devuelven los resultados de estas métricas en forma de diccionario.

In [15]:
metric = evaluate.load("accuracy")
metric

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [16]:
def compute_metrics_alone(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = (predictions == labels).mean()
    precision = precision_score(labels, predictions, average="binary")
    recall = recall_score(labels, predictions, average="binary")
    f1 = f1_score(labels, predictions, average="binary")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }



## Entrenamiento del modelo

Se establecen los parámetros para entrenar el modelo, como el tamaño de los lotes, el número de epochs de entrenamiento y la cantidad de datos utilizados para entrenar y evaluar. También se configura el aprendizaje, indicando cómo ajustar los pesos del modelo y las estrategias de evaluación para seleccionar el mejor modelo durante el proceso de entrenamiento, con un enfoque en la métrica de precisión.

In [19]:
model_name = model_checkpoint.split("/")[-1]

batch_size = 16
num_train_epochs= 4
num_train_samples = 10_000
num_eval_samples = 3_800
train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
eval_dataset = encoded_dataset["validation"].shuffle(seed=42).select(range(num_eval_samples))
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args_alone = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,
    learning_rate=1e-05,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Se configura un entrenador para el modelo, especificando los datos de entrenamiento y validación, así como el tokenizador y las métricas que se utilizarán para evaluar el rendimiento del modelo durante el proceso de entrenamiento. 

In [20]:
trainer_alone = Trainer(
    model=model,
    args=training_args_alone,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["validation"], 
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_alone
)

Se entrena el modelo durante varias épocas, mostrando los resultados en cada una de ellas.

In [21]:
trainer_alone.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.202400,0.313850,0.886126,0.799550,0.947298,0.867176
2,0.084300,0.331888,0.921204,0.934058,0.859907,0.895450
3,0.007800,0.391546,0.927225,0.904037,0.911274,0.907641
4,0.010500,0.406407,0.927487,0.920799,0.891928,0.906134


TrainOutput(global_step=2500, training_loss=0.1085736317721894, metrics={'train_runtime': 426.1863, 'train_samples_per_second': 93.856, 'train_steps_per_second': 5.866, 'total_flos': 659282304747840.0, 'train_loss': 0.1085736317721894, 'epoch': 4.0})

Se crea una métrica combinada para evitar el sobreajuste del modelo, ya que, como se observa, la pérdida en el conjunto de entrenamiento es muy baja a medida que aumentan las épocas, pero la pérdida en el conjunto de validación tiende a aumentar. Esta métrica combinada toma en cuenta tanto la exactitud como la pérdida de validación. Esta medida se usará como referencia para encontrar los mejores hiperparámetros.

In [22]:
for log in trainer_alone.state.log_history:
    if 'eval_loss' in log and 'eval_accuracy' in log:
        validation_loss = log['eval_loss']
        accuracy = log['eval_accuracy']
        combined_metric = accuracy - (0.5 * validation_loss)
        print(f"Validation Loss: {validation_loss}, Accuracy: {accuracy}, Combined Metric: {combined_metric}")

Validation Loss: 0.3138503134250641, Accuracy: 0.8861256544502618, Combined Metric: 0.7292004977377298
Validation Loss: 0.3318882882595062, Accuracy: 0.9212041884816754, Combined Metric: 0.7552600443519223
Validation Loss: 0.39154571294784546, Accuracy: 0.9272251308900523, Combined Metric: 0.7314522744161296
Validation Loss: 0.40640681982040405, Accuracy: 0.9274869109947644, Combined Metric: 0.7242835010845624


## Busqueda de hiperparámetros

Se utiliza la librería de optimización 'optuna' para ajustar automáticamente los hiperparámetros del modelo con el objetivo de maximizar su rendimiento. Entre los parámetros que se prueban están la tasa de aprendizaje, el tamaño del lote, el número de epochs y el valor de decaimiento de los pesos. A través de múltiples pruebas, esta técnica ayuda a encontrar la mejor combinación de valores para mejorar el rendimiento del modelo.

In [23]:
study = optuna.create_study(direction="maximize")

def suggest_hyperparameters(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32])
    num_epochs = trial.suggest_int("num_epochs", 1, 5)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.3)

    return {
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "num_epochs": num_epochs,
        "weight_decay": weight_decay
    }


[I 2024-10-07 14:49:28,442] A new study created in memory with name: no-name-0412446f-a79a-410f-9b62-c11cf5a9bd0c


Esta función define el proceso de entrenamiento del modelo de clasificación de secuencias. Recibe como argumento un conjunto de hiperparámetros optimizados, que se utilizan para configurar las condiciones bajo las cuales se entrenará el modelo.

Dentro de la función, también se especifica cómo se evaluará el modelo a lo largo de las épocas y cómo se gestionarán los resultados. Finalmente, aunque el modelo y el entrenador están listos, la función devuelve el entrenador preparado para que se ejecute en otro punto del código.

In [24]:
def train_model(hyperparams):
    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",
        save_strategy="no", 
        learning_rate=hyperparams["learning_rate"],
        per_device_train_batch_size=hyperparams["batch_size"],
        num_train_epochs=hyperparams["num_epochs"],
        weight_decay=hyperparams["weight_decay"],
        logging_steps=logging_steps
    )

    
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_alone
    )

    trainer.train()

    return trainer  

Esta función calcula una métrica combinada a partir del historial de entrenamiento del modelo. Busca el registro más reciente que contenga tanto la pérdida de validación como el accuracy. A partir de estos valores, calcula una métrica combinada, que equilibra el accuracy con la pérdida de validación, restando la mitad de la pérdida al accuracy. Esta métrica combinada permite evaluar el modelo de una manera más equilibrada y se imprime junto con los valores de pérdida y accuracy.

In [25]:
def calculate_combined_metric(trainer):
    last_log = None

    for log in reversed(trainer.state.log_history):
        if 'eval_loss' in log and 'eval_accuracy' in log:
            last_log = log
            break

    if last_log:
        validation_loss = last_log['eval_loss']
        accuracy = last_log['eval_accuracy']
        combined_metric = accuracy - (0.5 * validation_loss)
        print(f"Validation Loss: {validation_loss}, Accuracy: {accuracy}, Combined Metric: {combined_metric}")
        return combined_metric

    return None



Este bloque de código realiza la búsqueda de los mejores hiperparámetros utilizando Optuna. Se inicializan variables globales para almacenar la mejor métrica combinada y los mejores hiperparámetros. La función objective se encarga de obtener los hiperparámetros sugeridos, entrenar el modelo con esos parámetros, y luego calcular la métrica combinada.

Si la nueva métrica combinada es mejor que la anterior, se actualizan los valores de los mejores hiperparámetros y de la mejor métrica. Finalmente, se ejecuta el proceso de optimización, ajustando los hiperparámetros en varias pruebas para maximizar la métrica combinada, y se imprimen los mejores valores obtenidos.

In [26]:
best_combined_metric = -float('inf') 
best_hyperparams = None  

def objective(trial):
    global best_combined_metric, best_hyperparams
    hyperparams = suggest_hyperparameters(trial)
    trainer = train_model(hyperparams)
    combined_metric = calculate_combined_metric(trainer)
    if combined_metric is not None and combined_metric > best_combined_metric:
        best_combined_metric = combined_metric
        best_hyperparams = hyperparams
    return combined_metric

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10) 

print(f"Mejores hiperparámetros: {best_hyperparams}, Mejor métrica combinada: {best_combined_metric}")

[I 2024-10-07 14:49:28,472] A new study created in memory with name: no-name-536d1256-d352-42e7-b686-1bf3d500fcef
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.223200,0.321303,0.881053,0.791061,0.947791,0.862363
2,0.087700,0.311556,0.927368,0.904920,0.910977,0.907939
3,0.018200,0.374981,0.932895,0.931106,0.895582,0.912999
4,0.021400,0.406106,0.932105,0.924451,0.900937,0.912542


[I 2024-10-07 14:55:29,256] Trial 0 finished with value: 0.7290524145803954 and parameters: {'learning_rate': 3.0814595241199084e-05, 'batch_size': 16, 'num_epochs': 4, 'weight_decay': 0.19567002167204414}. Best is trial 0 with value: 0.7290524145803954.


Validation Loss: 0.4061056971549988, Accuracy: 0.9321052631578948, Combined Metric: 0.7290524145803954


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Validation Loss: 0.48773548007011414, Accuracy: 0.9302631578947368, Combined Metric: 0.6863954178596797


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.212600,0.194678,0.929211,0.904290,0.917001,0.910602
2,0.052600,0.244071,0.935789,0.941384,0.892236,0.916151


[I 2024-10-07 15:28:22,589] Trial 9 finished with value: 0.8137537765816638 and parameters: {'learning_rate': 2.9498580198388856e-05, 'batch_size': 32, 'num_epochs': 2, 'weight_decay': 0.21863411538200875}. Best is trial 9 with value: 0.8137537765816638.


Validation Loss: 0.24407139420509338, Accuracy: 0.9357894736842105, Combined Metric: 0.8137537765816638
Mejores hiperparámetros: {'learning_rate': 2.9498580198388856e-05, 'batch_size': 32, 'num_epochs': 2, 'weight_decay': 0.21863411538200875}, Mejor métrica combinada: 0.8137537765816638


## Entrenamiento del modelo con los parametros optimizados

Ahora se entrena el modelo utilizando los mejores hiperparámetros encontrados en la búsqueda previa. 

In [21]:
training_args_final = TrainingArguments(
    output_dir="final_results",
    num_train_epochs=best_hyperparams['num_epochs'],
    learning_rate=best_hyperparams['learning_rate'],
    per_device_train_batch_size=best_hyperparams['batch_size'],
    per_device_eval_batch_size=best_hyperparams['batch_size'],
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=best_hyperparams['weight_decay'],
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-spanish_sarcastic_texts"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:2026: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case Ernesto-1997/roberta-base-bne-finetuned-spanish_sarcastic_texts).
  warnings.warn(


In [22]:
final_trainer = Trainer(
    model=model,
    args=training_args_final,
    compute_metrics=compute_metrics_alone,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

In [23]:
final_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.219500,0.200254,0.925526,0.910508,0.898929,0.904682
2,0.061400,0.256512,0.932368,0.935257,0.889558,0.911835


TrainOutput(global_step=626, training_loss=0.1826324062444234, metrics={'train_runtime': 180.264, 'train_samples_per_second': 110.948, 'train_steps_per_second': 3.473, 'total_flos': 375911698125120.0, 'train_loss': 0.1826324062444234, 'epoch': 2.0})

Se sube el mejor modelo a Hugginface para su posterior uso

In [24]:
final_trainer.push_to_hub()

events.out.tfevents.1728316716.b7f92b4a7086.1464.0:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ernesto-1997/roberta-base-bne-finetuned-spanish_sarcastic_texts/commit/5038b905e59d1512d42f9465abc42dfe585e23b2', commit_message='End of training', commit_description='', oid='5038b905e59d1512d42f9465abc42dfe585e23b2', pr_url=None, pr_revision=None, pr_num=None)